<a href="https://colab.research.google.com/github/amrhsnd/ADM_HW3/blob/federico/ADM_hw3_point2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# i loaded the file
file_path = '/content/output.csv'
data = pd.read_csv(file_path)

data.head()


,index,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,creditCards,facilitiesServices,phoneNumber,website
0,1,20Tre,via David Chiossone 20 r,Genoa,16123,Italy,€€,"Farm to table, Modern Cuisine",Situated in the heart of Genoa’s historic cent...,"['Amex', 'Dinersclub', 'Mastercard', 'Visa']",['Air conditioning'],+39 010 247 6191,https://www.ristorante20tregenova.it/
1,2,Alessandro Feo,via Angelo Lista 24,Marina di Casal Velino,84040,Italy,€€,"Campanian, Seafood",In a beautiful stone-vaulted building (an old ...,"['Amex', 'Dinersclub', 'Mastercard', 'Visa']",[],+39 328 893 7083,https://www.alessandrofeoristorante.it/
2,3,Ape Vino e Cucina,Piazza Risorgimento 3,Alba,12051,Italy,€€,"Piedmontese, Contemporary",This attractive restaurant in the heart of Alb...,"['Amex', 'Dinersclub', 'Maestrocard', 'Masterc...","['Air conditioning', 'Terrace', 'Wheelchair ac...",+39 0173 363453,https://www.apewinebar.it/alba/
3,4,Charleston,via Generale Magliocco 19,Palermo,90141,Italy,€€€€,"Modern Cuisine, Creative","Before it became famous in Mondello, the renow...","['Amex', 'Mastercard', 'Visa']","['Air conditioning', 'Counter dining', 'Terrac...",+39 091 450171,https://casacharleston.net/
4,5,Da Bob Cook Fish,largo Parsano vecchio 16,Sorrento,80067,Italy,€€,Seafood,Working in partnership with the nearby fishmon...,"['Amex', 'Dinersclub', 'Mastercard', 'Visa']","['Air conditioning', 'Terrace']",+39 081 1778 3873,https://www.dabobcookfish.com/


#**[2.0]**

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

# Download stopwords
nltk.download('stopwords')

def preprocess_text(text):
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()

    # Convert text to lowercase and remove punctuation including specific quotes and numbers
    text = text.lower().translate(str.maketrans("", "", '‘’“”-1234567890' + string.punctuation))

    # Tokenize, remove stopwords and stem (and remove single-letter words)
    tokens = [stemmer.stem(word) for word in text.split() if word not in stop_words and len(word) > 1]

    return " ".join(tokens)

# Apply preprocessing to descriptions
data["processed_description"] = data["description"].apply(preprocess_text)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#**[2.1]**

2.1.1

In [ ]:
from collections import defaultdict
#vocabulary and inverted index
vocabulary = {}
inverted_index = defaultdict(list)

term_id = 0

for idx, row in data.iterrows():
    description = row["processed_description"]
    restaurant_id = row[" index"]  # The variable is called ' index' in my csv file

    for word in set(description.split()):  # Use set to avoid duplicates in one description
        if word not in vocabulary:
            vocabulary[word] = term_id
            term_id += 1

        term_id_for_word = vocabulary[word]
        inverted_index[term_id_for_word].append(restaurant_id)


In [ ]:
import json

# Save vocabulary as vocabulary.csv
vocab_df = pd.DataFrame(list(vocabulary.items()), columns=["word", "term_id"])
vocab_df.to_csv("/content/vocabulary.csv", index=False)

# Save inverted index as inverted_index.json
with open("/content/inverted_index.json", "w") as f:
    json.dump(inverted_index, f)


2.1.2

In [ ]:
# Function to preprocess and execute the query
def execute_query(query, data):
    # Preprocess the query terms using the preprocess_text function
    processed_query = preprocess_text(query)
    query_terms = set(processed_query.split())  # Use a set for faster lookup

    # Find matching restaurants by checking if all query terms are in the processed description
    matched_restaurants = data[data["processed_description"].apply(lambda desc: query_terms.issubset(desc.split()))]

    # SSelect only the necessary columns for the output
    return matched_restaurants[["restaurantName", "address", "description", "website"]]

# Prompt the user to input a query
query = input("Please enter your search terms (e.g., 'modern seasonal cuisine'): ")

result = execute_query(query, data)

result


Please enter your search terms (e.g., 'modern seasonal cuisine'): modern seasonal cuisine


,restaurantName,address,description,website
24,Casin del Gamba,via Roccolo Pizzati 1,The journey to get here – a winding road through woods and hills – may be challenging at times b...,https://www.casindelgamba.it/
98,San Giorgio,viale Brigate Bisagno 69r,"Situated in the city albeit not right in the centre, San Giorgio is a typical Genovese restauran...",https://www.ristorantesangiorgiogenova.it/
186,Il Luogo Aimo e Nadia,via Montecuccoli 6,This long-established restaurant has been part of the Milanese culinary scene for over 60 years ...,https://www.aimoenadia.com/il-luogo-aimo-e-nadia
211,Vesta Mare,viale Roma 41,"This typical, elegant Versilian beach club with an open-plan feel extending from the car park to...",https://vestafiorichiari.com/mare/
302,Ca' Del Moro,località Erbin 31,"Situated within the La Collina dei Ciliegi wine estate amid the verdant Valpantena hills, this f...",https://www.cadelmoro.wine/it
323,Contrasto,via Roma 55,"Having returned to his native village, owner-chef Lucio Testa has opened this restaurant in a fo...",https://contrastoristorante.it
336,Saur,via Filippo Turati 8,"In a tiny rural village, this contemporary, almost minimalist-style restaurant serves modern cui...",https://ristorantesaur.it
416,San Michele,via Castello di Fagagna 33,"Situated next to the ruins of the old castle and the small church of San Michele, this 13C build...",http://sanmichele.restaurant
446,Chichibio,via Guglielmo Marconi 1,"Despite its lack of awards, this restaurant stands out for the quality of its cuisine. Situated ...",tel:+39 328 905 4831
499,Winter Garden Florence,piazza Ognissanti 1,"Horse-drawn carriages once entered the old courtyard of the St Regis hotel, now converted into a...",https://www.wintergardenflorence.com/it/
